**LING 484: Final Project**

Duygu Demiray & Elifnur Ulusoy (Group 5)

Raincheck: a weather app which tells you what to wear/pack

Important note: this does NOT work in Google Colab because we have to take input from a mic. Please run it on your preferred environment. We used Jupyter Notebooks. 

Step 1: Listen to user commands (I know voice commands are not 
necessary, but I still wanted to give it a try)
[source](https://sweetcode.io/how-build-digital-virtual-assistant-python/)

In [1]:
import re
import speech_recognition as sr 
import pyaudio
from time import ctime
import time
import os
! pip install gtts
from gtts import gTTS
import requests, json
!pip install flair
from flair.data import Sentence
from flair.models import SequenceTagger

You should consider upgrading via the 'c:\users\duygu\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\duygu\miniconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
def listen():
   
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("I am listening...")
        audio = r.listen(source)
    data = ""
    try:
        data = r.recognize_google(audio)
        print("You said: " + data)
    except sr.UnknownValueError:
        print("I don't understand. Could you repeat that?")
    except sr.RequestError as e:
        print("Sorry, I don't understand.; {0}".format(e))
    return data

Framework used for named entity recognition: https://github.com/flairNLP/flair

In [3]:
def Tagger(data):
    """Performs entity recognition using the package Flair"""
    sentence = Sentence(data)
    tagger = SequenceTagger.load('ner-ontonotes-fast')
    tagger.predict(sentence)
    tag_dict = sentence.to_dict(tag_type = 'ner')
    tag_dict_entities = tag_dict['entities']
    entities_list = []
    #now we need to accesss each dictionary in the list tag_dict_entities, and retrieve the 
    #key 'labels'. 
    for i in range(len(tag_dict_entities)):
        entities_list.append(tag_dict_entities[i])
    
    for j in range(len(entities_list)):
        dic = entities_list[j]
        del dic['start_pos']
        del dic['end_pos']
        
    return entities_list

#this package does not tag stuff like afternoon :)))))

What Tagger currently returns is sth. like this: 
[{'text': 'next week', 'labels': [DATE (0.7448)]},
 {'text': 'France', 'labels': [GPE (0.9999)]}]
 
Let's do this with regex
 

A problem with the function below is that it will definitely overgenerate. The sequence tagger used inside Tagger() can tag 18 classes, whereas we are only interested in the following classes:

* DATE: date value
* GPE: geo-political entity 
* LOC: location name
* TIME: time value

Actually, we only need TIME at this point. We can try to get location-based weather input and then we can use GPE & LOC too. We will need to pull some tricks to use DATE too but I don't really think that is necessary. 

We can modify Tagger() such that it only returns sentences w/ time & location information and throws some sort of error message otherwise. 

In [4]:
def ReturnDic():
    """Takes voice input"""
    time.sleep(2)
    listening = True
    while listening == True:
        data = listen()
        listening = Tagger(data)
    return listening

In [5]:
def Purge ():
    """Labels relevant info as value 1, and non-relevant as value 0"""
    list = ReturnDic()
    pattern_date = re.compile("DATE")
    pattern_time = re.compile("TIME")
    pattern_gpe = re.compile("GPE")
    pattern_loc = re.compile("LOC")
    for i in range(len(list)):
        dic = list[i]
        labels_string = str(dic['labels'])
        matches_date = re.findall(pattern_date, labels_string)
        matches_time = re.findall(pattern_time, labels_string)
        matches_gpe = re.findall(pattern_gpe, labels_string)
        matches_loc = re.findall(pattern_loc, labels_string)
        if matches_date == [] and matches_time == [] and matches_gpe == [] and matches_loc == []:
            dic["value"] = 0
        elif matches_date == [] and matches_time == [] and matches_gpe == []:
            dic["value"] = 1 #location tag
        elif matches_date == [] and matches_time == [] and matches_loc == []:
            dic["value"] = 2 #GPE tag
        elif  matches_date == [] and matches_gpe == [] and matches_loc == []:
            dic["value"] = 3 #time tag
        else:
            dic["value"] = 4 #date tag
    return list

In [22]:
x = Purge()

I am listening...
You said: weather tomorrow 12 p.m.
2021-06-30 18:05:39,117 --------------------------------------------------------------------------------
2021-06-30 18:05:39,135 The model key 'ner-ontonotes-fast' now maps to 'https://huggingface.co/flair/ner-english-ontonotes-fast' on the HuggingFace ModelHub
2021-06-30 18:05:39,138  - The most current version of the model is automatically downloaded from there.
2021-06-30 18:05:39,138  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-ontonotes-fast/en-ner-ontonotes-fast-v0.4.pt)
2021-06-30 18:05:39,139 --------------------------------------------------------------------------------
2021-06-30 18:05:40,087 loading file C:\Users\duygu\.flair\models\ner-english-ontonotes-fast\0d55dd3b912da9cf26e003035a0c269a0e9ab222f0be1e48a3bbba3a58c0fed0.c9907cd5fde3ce84b71a4172e7ca03841cd81ab71d13eb68aa08b259f57c00b6


In [23]:
x

[{'text': 'tomorrow', 'labels': [DATE (0.9431)], 'value': 4},
 {'text': '12 p.m', 'labels': [TIME (0.9365)], 'value': 3}]

some questions:
*   do we need multiple locations? should we let the user choose their location? (if so, find out how)

This will be difficult because the URL we are reading specifically fetches the weather for istanbul. I do not know how to go into the website AND change the location based on the users location. I think it might be a bit too much work, and we might need to read the website differently. We can get around to it if we have time. - D.



Step 2: Read into the Weather Channel website and get Today's Forecast and Daily Forecast. 

In [7]:
#elif's version of duygu's code -- to find out whether we can switch btwn F/C
#the code below only prints in C but doesn't really matter since it only shows weather for Istanbul

import urllib.request 

def urlopen():

    class AppURLopener(urllib.request.FancyURLopener): 
        version = "Mozilla/5.0"

    opener = AppURLopener()
    response = opener.open('https://weather.com/tr-TR/weather/today/l/33d1e415eb66f3e1ab35c3add45fccf4512715d329edbd91c806a6957e123b49')
    file = response.read().decode('utf-8') 

    pattern = re.compile("([0-9]+)°") 

    matches = re.findall(pattern, file) 

    return matches #returns in celsius

matches is a list of numbers, and maybe indexing them will be easier than writing separate RegEx's. we can create a dictionary and then use that dictionary in the 3rd step.

this is a very dumb way and we're not doing much work w/ regex but it works, so i think we'll be fine. 

beware: I have tweaked with CleanList because of the website. if it does not work for you in the dictionary function below make while a<2 and while b<=10. 

In [9]:
def CleanList():
    """Returns the relevant numbers from the scraped data""" 

    matches = urlopen()
    a=0
    while a<1:
        matches.pop(1)
        a+=1

    b=0
    while b<=8:
        matches.pop(5)
        b+=1
            
    return matches


In [10]:
#we are turning them to int's. for some reason this did not work when I put it inside CleanList
def turntoint():
    
    list = Cleanlist()
    integer_map = map(int, list) 
    matches_int = list(integer_map)

    return matches_int

open the weather channel and look side-by-side.

[0-4] -> 0. current weather / 1. morning / 2. afternoon / 3. evening / 4. overnight 

[5-12] -> skip down to daily forecast in the website, which gives us the weather for the next four days. I kept the values for both day & night, but of course we can purge it further. 

now we need to match these with the list indeces. 

In [11]:
#maybe we should get rid of everything that comes after day_after_n. 
def CreateDictionary():
    """Creating a dictionary from the numbers obtained from the weather channel and times of the day"""
    
    labels = ["cur_weather", "morning", "afternoon", "evening", "overnight", "tomorrow", "tomorrow_night", "day_after", "day_after_n", "after_after", "after_after_n", "idk", "idk_n"]
    matches = turntoint()
    weather = {}

    for k in range(len(labels)):
        weather[labels[k]] = matches[k]
        
    return weather

Some problems with this: we are taking everything but we will possibly only use a limited part of it when we get input from the user, as they will ask what to wear for a specific time of the day/week. It would be better to have separate RegEx's that read the relevant parts of the website based on user input, but I couldn't think of/find a way to do that. 

[{'text': 'next week', 'labels': [DATE (0.7448)], 'value': 1},
 {'text': 'France', 'labels': [GPE (0.9999)]}, 'value': 1]
 
 1: location
 2: GPE
 3: time
 4: date

In [41]:
def GenerateOutput():
    
    list = Purge()
    weather_dictionary = CreateDictionary()
   
    time_pattern_1 = re.compile("at [(0-9:) (ampm)]+")
    time_date_pattern = re.compile("([a-z]+) on (\d+)[.//-](\d+)[//.-](\d+) at (\d+):?(\d+)?(pm)?\s?(am)?")
    time_date_pattern_1 = re.compile("([a-z]+) on (\w+) (\d+) (\d+) at (\d+)(pm)?:?(\d+)?")
    
    
    matches_time_date_pattern = 
    matches_time_date_pattern_1 = 
    
    for i in range(len(list)):
        dic = list[i]
        if 'value' in dic = 0:
            #raise error?
        elif 'value' in dic = 1: #location
            #we need to use api to fill this
        elif 'value' in dic = 2: #GPE
            #we need to use api to fill this
        elif 'value' in dic = 3: #time
            matches_time_pattern_1 = re.findall(time_pattern_1, dic['text'])
            int_matches = int(matches_time_pattern_1[0])
            if matches_time_pattern_1[1] == 'am':
                if 6<=int_matches<12:
                    weather = weather_dictionary['morning']
                else: 
                    #this is an odd time period, just print out something accordingly
            elif matches_time_pattern == 'pm':
                if 1<=int_matches<
        else: #date
            #we need to use api for this too.     

I am listening...
You said: weather next week in France
2021-06-14 17:26:24,265 --------------------------------------------------------------------------------
2021-06-14 17:26:24,265 The model key 'ner-ontonotes-fast' now maps to 'https://huggingface.co/flair/ner-english-ontonotes-fast' on the HuggingFace ModelHub
2021-06-14 17:26:24,265  - The most current version of the model is automatically downloaded from there.
2021-06-14 17:26:24,265  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-ontonotes-fast/en-ner-ontonotes-fast-v0.4.pt)
2021-06-14 17:26:24,273 --------------------------------------------------------------------------------
2021-06-14 17:26:25,038 loading file C:\Users\duygu\.flair\models\ner-english-ontonotes-fast\0d55dd3b912da9cf26e003035a0c269a0e9ab222f0be1e48a3bbba3a58c0fed0.c9907cd5fde3ce84b71a4172e7ca03841cd81ab71d13eb68aa08b259f57c00b6


NameError: name 'GPE' is not defined